In [1]:
!git clone https://github.com/timothycao/agnews-classifier.git
%cd agnews-classifier

Cloning into 'agnews-classifier'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 37 (delta 17), reused 28 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (37/37), 10.68 KiB | 3.56 MiB/s, done.
Resolving deltas: 100% (17/17), done.
/kaggle/working/agnews-classifier


In [2]:
# Model

from peft import LoraConfig
from model import create_lora_model

lora_config = LoraConfig(
    r=1,
    lora_alpha=2,
    lora_dropout=0.1,
    bias='none',
    target_modules=['query', 'key', 'value'],
    task_type='SEQ_CLS',
    use_rslora = False # regular LoRA: lora_alpha/r, rs-LoRA: lora_alpha/sqrt(r) -> normalization technique
)

model = create_lora_model(lora_config)

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 759,556 || all params: 125,408,264 || trainable%: 0.6057


In [3]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): Mod

In [5]:
# Training

from transformers import TrainingArguments
from train import main as train

training_args = TrainingArguments(
    # Core training configs
    max_steps=3000,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    optim='adamw_torch',
    learning_rate=5e-5,
    lr_scheduler_type='cosine',
    
    # Logging, evaluation, and checkpointing
    output_dir='/kaggle/working/saved_models',
    logging_strategy='steps',
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=100,
    save_strategy='steps',
    save_steps=100,

    # Miscellaneous
    report_to='none',
    dataloader_num_workers=4,
    gradient_checkpointing=False,
    gradient_checkpointing_kwargs={'use_reentrant':True}
)

train(model, training_args)

Starting training...


Step,Training Loss,Validation Loss,Accuracy
100,0.269000,0.306359,0.892188
200,0.281800,0.326250,0.889062
300,0.280900,0.314796,0.904687
400,0.245000,0.309985,0.906250
500,0.257400,0.317893,0.903125
600,0.240100,0.321106,0.907813
700,0.260600,0.300190,0.912500
800,0.270800,0.302510,0.904687
900,0.253700,0.300990,0.903125
1000,0.248600,0.300539,0.907813


In [9]:
training_args = TrainingArguments(
    # Core training configs
    max_steps=4000,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    optim='adamw_torch',
    learning_rate=5e-5,
    lr_scheduler_type='linear',
    
    # Logging, evaluation, and checkpointing
    output_dir='/kaggle/working/saved_models',
    logging_strategy='steps',
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=100,
    save_strategy='steps',
    save_steps=100,

    # Miscellaneous
    report_to='none',
    dataloader_num_workers=4,
    gradient_checkpointing=False,
    gradient_checkpointing_kwargs={'use_reentrant':True}
)

train(model, training_args, checkpoint='/kaggle/working/saved_models/checkpoint-3000')

Starting training...
Resuming from checkpoint: /kaggle/working/saved_models/checkpoint-3000


/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3418: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(checkpoint, OPTIMIZER_NAME), map_

Step,Training Loss,Validation Loss,Accuracy
3100,0.279400,0.283299,0.910937
3200,0.266500,0.285942,0.915625
3300,0.237500,0.289334,0.912500
3400,0.227600,0.291630,0.917188
3500,0.269200,0.289263,0.914062
3600,0.280100,0.288613,0.915625
3700,0.251200,0.287335,0.917188
3800,0.238500,0.286974,0.917188
3900,0.272200,0.287824,0.918750
4000,0.287000,0.287646,0.921875


In [6]:
# Inference

from inference import main as inference

data_path = '/kaggle/input/deep-learning-spring-2025-project-2/test_unlabelled.pkl'
checkpoint = '/kaggle/working/saved_models/checkpoint-700'
output_dir = '/kaggle/working/saved_predictions'

inference(data_path, checkpoint, output_dir)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running inference...


100%|██████████| 250/250 [00:59<00:00,  4.23it/s]

Predictions saved to /kaggle/working/saved_predictions/predictions_checkpoint-700.csv


In [7]:
import pandas as pd

csv_path = '/kaggle/working/saved_models/checkpoint-3000/processed_log_history.csv'

df = pd.read_csv(csv_path)
df

,Step,Train Loss,Test Loss,Train Acc,Test Acc,Learning Rate,Epochs,Loss Spread,Loss Ratio,Acc Spread,Acc Ratio
0,100,0.2690,0.306359,0.912500,0.892188,4.986305e-05,0.013405,-0.037359,0.878054,0.020312,1.022767
1,200,0.2818,0.326250,0.904911,0.889062,4.945369e-05,0.026810,-0.044450,0.863755,0.015848,1.017826
2,300,0.2809,0.314796,0.900893,0.904687,4.877641e-05,0.040214,-0.033896,0.892323,-0.003795,0.995806
3,400,0.2450,0.309985,0.915179,0.906250,4.783864e-05,0.053619,-0.064985,0.790361,0.008929,1.009852
4,500,0.2574,0.317893,0.909375,0.903125,4.665064e-05,0.067024,-0.060493,0.809706,0.006250,1.006920
5,600,0.2401,0.321106,0.916964,0.907813,4.522542e-05,0.080429,-0.081006,0.747729,0.009152,1.010081
6,700,0.2606,0.300190,0.913393,0.912500,4.357862e-05,0.093834,-0.039590,0.868117,0.000893,1.000978
7,800,0.2708,0.302510,0.905804,0.904687,4.172827e-05,0.107239,-0.031710,0.895177,0.001116,1.001234
8,900,0.2537,0.300990,0.909821,0.903125,3.969463e-05,0.120643,-0.047290,0.842886,0.006696,1.007415
9,1000,0.2486,0.300539,0.910714,0.907813,3.750000e-05,0.134048,-0.051939,0.827181,0.002902,1.003196
